In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim,GoogleV3
from sklearn.covariance import GraphicalLasso
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns
from uszipcode import SearchEngine

from scipy import stats

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
search = SearchEngine(simple_zipcode=True)

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


# Data Import

In [14]:
df=pd.read_csv('data/assessments.csv')
print(df.isnull().sum().sort_values(ascending=False))
df.info()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (19,20,29,30,37,38,46,57,83) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
FARMSTEADFLAG           580991
ABATEMENTFLAG           580419
CLEANGREEN              579294
TAXSUBCODE_DESC         579283
TAXSUBCODE              579283
ALT_ID                  554788
PREVSALEPRICE2          377774
PREVSALEDATE2           377191
HOMESTEADFLAG           283282
LEGAL3                  243139
RECORDDATE              232556
PREVSALEPRICE           202908
PREVSALEDATE            201214
FIREPLACES              169742
BSMTGARAGE              155299
HALFBATHS               138829
ROOFDESC                135688
ROOF                    135688
HEATINGCOOLING          135139
HEATINGCOOLINGDESC      135139
FULLBATHS               135082
BASEMENT                135060
BASEMENTDESC            135060


# Missing Data Conversions

In [15]:
def deleteAllKeyWords(df_in, keywords):
    '''
    Description:
        Searches for keywords in dataframe keys and removes them. If 'DESC+' is 
        a keyword, then all keys containing 'DESC' will be removed. 
    Input: 
        df_in - (Pandas DataFrame) containing data and all features
        keywords - (list) cotaining strings of keywords to be removed, if a '+' is included
        in keyword all keywords conating the string will be removed
    Output:
        (Pandas DataFrame) with removed data based on keywords 
    '''
    assert isinstance(df_in, pd.DataFrame), 'Wrong type for first input -- should be Pandas dataframe'   
    assert isinstance(keywords, list), 'Wrong type for second input -- should be list'   

    #ITERATE OVER ALL KEYWORDS
    for word in keywords:
        #INITIALIZE FLAG FOR KEYWORD AS FALSE
        searchFlag=False
        #CHECK IF KEYWORD INCLUDES THE A '+' IN THE STRING

        if word.find('+'):
            #IF SO, REMOVE PLUS AND SET FLAT TO TRUE
            word=word.replace('+','')
            searchFlag=True

        #ITERATE OVER KEYS IN THE DATAFRAME
        for key in df_in:
            #IF IT FLAG IS TRUE SEE IF KEYWORD CONTAINS WORD
            if searchFlag:
                if key.upper().find(word.upper()) !=-1:
                    df_in=df_in.drop(columns=[key])

            #IF NO FLAG, THEN JUST REMOVE KEYWORD EQUAL TO STRING
            else:
                df_in=df_in.drop(columns=[key])
    return df_in

def convertMissingToBoolean(df_in,keys):
    '''
    Description:
        Converts data into boolean feature. If data is null, the entry is set to False and if an entry exists, the 
        entry is set to True
    Input: 
        df_in - (Pandas DataFrame) containing complete data with missing values
        keys - (list) cotaining strings of dataframe keys to fix missing values accoringly
    Output:
        (Pandas DataFrame) with corrected missing values 
    '''
    assert isinstance(df_in, pd.DataFrame), 'Wrong type for first input -- should be Pandas dataframe'   
    assert isinstance(keys, list), 'Wrong type for second input -- should be list' 

    for key in keys:
        df_in[key]=~df_in[key].isnull()
    return df_in

def convertMissingToUnknown(df_in,keys):
    '''
    Description:
        Converts missing data into string containing 'U' for unknown.
    Input: 
        df_in - (Pandas DataFrame) containing complete data with missing values
        keys - (list) cotaining strings of dataframe keys to fix missing values accoringly
    Output:
        (Pandas DataFrame) with corrected missing values 
    '''
    assert isinstance(df_in, pd.DataFrame), 'Wrong type for first input -- should be Pandas dataframe'   
    assert isinstance(keys, list), 'Wrong type for second input -- should be list'

    for key in keys:
        df_in[key][df_in[key].isnull()]='U'
    return df_in

def convertMissingToZero(df_in,keys):
    '''
    Description:
        Converts missing data into value containing 0 for unknown.
    Input: 
        df_in - (Pandas DataFrame) containing complete data with missing values
        keys - (list) cotaining strings of dataframe keys to fix missing values accoringly
    Output:
        (Pandas DataFrame) with corrected missing values 
    '''
    assert isinstance(df_in, pd.DataFrame), 'Wrong type for first input -- should be Pandas dataframe'   
    assert isinstance(keys, list), 'Wrong type for second input -- should be list'

    for key in keys:
        df_in[key][df_in[key].isnull()]=0
    return df_in

def convertMissingToMean(df_in,keys):
    '''
    Description:
        Converts missing data into value containing mean of data.
    Input: 
        df_in - (Pandas DataFrame) containing complete data with missing values
        keys - (list) cotaining strings of dataframe keys to fix missing values accoringly
    Output:
        (Pandas DataFrame) with corrected missing values 
    '''
    assert isinstance(df_in, pd.DataFrame), 'Wrong type for first input -- should be Pandas dataframe'   
    assert isinstance(keys, list), 'Wrong type for second input -- should be list'

    for key in keys:
        df_in[key][df_in[key].isnull()]=df_in[key].mean()
    return df_in

def convertMissingToMeanInt(df_in,keys):
    '''
    Description:
        Converts missing data into value containing closest mean integer of data.
    Input: 
        df_in - (Pandas DataFrame) containing complete data with missing values
        keys - (list) cotaining strings of dataframe keys to fix missing values accoringly
    Output:
        (Pandas DataFrame) with corrected missing values 
    '''
    assert isinstance(df_in, pd.DataFrame), 'Wrong type for first input -- should be Pandas dataframe'   
    assert isinstance(keys, list), 'Wrong type for second input -- should be list'
    for key in keys:
        df_in[key][df_in[key].isnull()]=int(df_in[key].mean())
    return df_in

def replaceMissingFromDifferentColumn(df_in,key,getkey):
    '''
    Description:
        Converts missing data into value from another column.
    Input: 
        df_in - (Pandas DataFrame) containing complete data with missing values
        key - (string) of column to be replaced
        getkey - (string) of column to get replacement value from
    Output:
        (Pandas DataFrame) with corrected missing values 
    '''
    assert isinstance(df_in, pd.DataFrame), 'Wrong type for first input -- should be Pandas dataframe'   
    assert isinstance(key, str), 'Wrong type for second input -- should be string'
    assert isinstance(getkey, str), 'Wrong type for third input -- should be string'

    df_in[key]=df_in[key].fillna(df_in[getkey])
    return df_in

def combineDataAndRemove(df_in,keys,newKey):
    '''
    Description:
        Combines data from different columns and delets original column
    Input: 
        df_in - (Pandas DataFrame) containing complete data
        keys - (list) list of strings containing keys of columns column to be replaced in order provided
        newkey - (string) name of new key for converted data
    Output:
        (Pandas DataFrame) with corrected values 
    '''
    assert isinstance(df_in, pd.DataFrame), 'Wrong type for first input -- should be Pandas dataframe'   
    assert isinstance(keys, list), 'Wrong type for second input -- should be list'
    assert isinstance(newKey, str), 'Wrong type for third input -- should be string'

    df_in[newKey]=df_in[keys].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
    df_in=df_in.drop(columns=keys)
    return df_in

def convertDateToDateTime(df_in,keys):
    '''
    Description:
        Converts date string to datetime type
    Input: 
        df_in - (Pandas DataFrame) containing complete data
        keys - (list) list of strings containing keys of columns column to be converted
    Output:
        (Pandas DataFrame) with converted values 
    '''
    assert isinstance(df_in, pd.DataFrame), 'Wrong type for first input -- should be Pandas dataframe'   
    assert isinstance(keys, list), 'Wrong type for second input -- should be list'

    for key in keys:
        df_in[key]=pd.to_datetime(df_in[key], format='%m-%d-%Y', errors = 'coerce')
        df_in=df_in.dropna(subset=[key])
    return df_in

In [16]:
df2=df.copy()

#DROP ALL DATA TAHT HAVE MISSING IMPORTANT INFORMATION 
df2=df2.dropna(how='all')
df2=df2.dropna(subset=['SALEPRICE', 'SALEDATE','PROPERTYHOUSENUM','PROPERTYCITY','PROPERTYZIP'])
df2=df2[df2['SALEPRICE'] > 1000]


#DELETE KEYWORDS THAT ARE UNIMPORTANT SUCH AS DESCRIPTIONS AND ALTERNATE REPETITIVE INFORMATION
df2=deleteAllKeyWords(df2,['DEED+','DESC+','CHANGENOTICE+','ALT_ID','RECORDDATE','LEGAL+','TAXYEAR'])

#CONVERT MISSING INFORMATION DEPENDING ON EACH OF THE FEATURES
df2=convertMissingToBoolean(df2,['FARMSTEADFLAG','ABATEMENTFLAG','CLEANGREEN','HOMESTEADFLAG'])
df2=convertMissingToUnknown(df2,['TAXSUBCODE','CDU','GRADE'])
df2=convertMissingToZero(df2,['EXTERIORFINISH','BSMTGARAGE','ROOF','HEATINGCOOLING','BASEMENT','CONDITION','CARDNUMBER','STYLE','SALECODE'])
df2=convertMissingToMeanInt(df2,['FIREPLACES','FULLBATHS','HALFBATHS','YEARBLT','TOTALROOMS','BEDROOMS','STORIES'])
df2=convertMissingToMean(df2,['FINISHEDLIVINGAREA'])

#REPLACE INFORMATION FROM DIFFERENT COLUMNS IF MISSING
df2=replaceMissingFromDifferentColumn(df2,'PREVSALEDATE','SALEDATE')
df2=replaceMissingFromDifferentColumn(df2,'PREVSALEPRICE','SALEPRICE')

df2=replaceMissingFromDifferentColumn(df2,'PREVSALEDATE2','PREVSALEDATE')
df2=replaceMissingFromDifferentColumn(df2,'PREVSALEPRICE2','PREVSALEPRICE')

#COMBINE INFORMATION AND DELETE REPETITIVE
df2=combineDataAndRemove(df2,['PROPERTYHOUSENUM','PROPERTYADDRESS','PROPERTYCITY','PROPERTYSTATE'],'address')

#CONVERT TO APPROPRIATE DATATYPE
df2=convertDateToDateTime(df2,['SALEDATE','PREVSALEDATE','PREVSALEDATE2'])

# df2.to_pickle("./cleanData2.pkl")

print(df2.info())

df2.isnull().sum().sort_values(ascending=False)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/shared-libs/python3.7/py-core/lib/python3.7/site-p

PARID                 0
GRADE                 0
LOCALLAND             0
LOCALTOTAL            0
FAIRMARKETBUILDING    0
FAIRMARKETLAND        0
FAIRMARKETTOTAL       0
STYLE                 0
STORIES               0
YEARBLT               0
EXTERIORFINISH        0
ROOF                  0
BASEMENT              0
CONDITION             0
PROPERTYFRACTION      0
CDU                   0
TOTALROOMS            0
BEDROOMS              0
FULLBATHS             0
HALFBATHS             0
HEATINGCOOLING        0
FIREPLACES            0
BSMTGARAGE            0
FINISHEDLIVINGAREA    0
CARDNUMBER            0
ASOFDATE              0
LOCALBUILDING         0
COUNTYEXEMPTBLDG      0
COUNTYTOTAL           0
COUNTYLAND            0
PROPERTYUNIT          0
PROPERTYZIP           0
MUNICODE              0
SCHOOLCODE            0
NEIGHCODE             0
TAXCODE               0
TAXSUBCODE            0
OWNERCODE             0
CLASS                 0
USECODE               0
LOTAREA               0
HOMESTEADFLAG   

# Geo Location For Properties

In [12]:
df3=pd.read_pickle("./cleanData.pkl")
df3=df3.sample(10000)

locator = Nominatim(user_agent='myGeocoder')
# g = GoogleV3() # much faster but costs money

ct=0
def convert(x):
    global ct
    ct+=1
    if ct%100==0:
        print('At {}'.format(ct))
    location = locator.geocode(x)
    try:
        return (location.latitude, location.longitude)
    except:
        return None

df3['location'] = df3['address'].apply(convert)
# df3.to_pickle('./cleanData_geocode.pkl')

At 100


# Geolocation of Zipcodes

In [6]:
df3=pd.read_pickle("./cleanData2.pkl")
# df3=df3.sample(100)

zipMap={}
allZips=df3['PROPERTYZIP'].unique()

for i in allZips:
    location = search.by_zipcode(int(i))
    zipMap[str(int(i))]=(location.lng, location.lat)


df3['latlng-zip']=df3['PROPERTYZIP'].apply(lambda x: zipMap[str(int(x))])
df3['lng-zip']=df3['latlng-zip'].apply(lambda x: x[0])
df3['lat-zip']=df3['latlng-zip'].apply(lambda x: x[1])
df3=df3.dropna(subset=['lat-zip', 'lng-zip'])

# df3.to_pickle('./cleanData_geocode_zip.pkl')

# Variable Types Corrections

In [7]:
df=pd.read_pickle("./cleanData_geocode_zip.pkl")

In [8]:
df2=df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378562 entries, 11 to 580996
Data columns (total 57 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   PARID               378562 non-null  object        
 1   PROPERTYFRACTION    378562 non-null  object        
 2   PROPERTYUNIT        378562 non-null  object        
 3   PROPERTYZIP         378562 non-null  float64       
 4   MUNICODE            378562 non-null  int64         
 5   SCHOOLCODE          378562 non-null  int64         
 6   NEIGHCODE           378562 non-null  object        
 7   TAXCODE             378562 non-null  object        
 8   TAXSUBCODE          378562 non-null  object        
 9   OWNERCODE           378562 non-null  int64         
 10  CLASS               378562 non-null  object        
 11  USECODE             378562 non-null  int64         
 12  LOTAREA             378562 non-null  int64         
 13  HOMESTEADFLAG       378562 n

In [9]:
from sklearn.preprocessing import OrdinalEncoder

#DELETING USELESS COLUMNS
df2=df2.drop(columns=['NEIGHCODE','latlng-zip','address','ASOFDATE'])
#CONVERTING OBJECTS INTO CATEGORICAL INTEGERS
cols = df2.select_dtypes('object').columns
df2[cols] = df2[cols].stack().astype('category').cat.codes.unstack()
# ord_enc = OrdinalEncoder()
# df2[cols] = ord_enc.fit_transform(df2[cols])

#CONVERT SELECTED DATA INTO CATEGORICAL INTEGERS
cols=['ROOF','BASEMENT','EXTERIORFINISH','CONDITION']
df2[cols] = df2[cols].stack().astype('category').cat.codes.unstack()

#CONVERT INT64 TO INT32 FOR MEMORY EFFICIENCY
cols = df2.select_dtypes('int64').columns
df2[cols] = df2[cols].astype('int32')

df2['pricediff']=df2['SALEPRICE']-df2['PREVSALEPRICE']
df2['pricediff2']=df2['PREVSALEPRICE']-df2['PREVSALEPRICE2']

df2['pricediffmax']=df2[['pricediff','pricediff2']].max(axis=1)

df2['saledate-int']=mdates.date2num(df2['SALEDATE'])
df2['prevsaledate-int']=mdates.date2num(df2['PREVSALEDATE'])
df2['prevsaledate2-int']=mdates.date2num(df2['PREVSALEDATE2'])


# df2.to_pickle("./cleanData_model.pkl")

In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378562 entries, 11 to 580996
Data columns (total 59 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   PARID               378562 non-null  int32         
 1   PROPERTYFRACTION    378562 non-null  int32         
 2   PROPERTYUNIT        378562 non-null  int32         
 3   PROPERTYZIP         378562 non-null  float64       
 4   MUNICODE            378562 non-null  int32         
 5   SCHOOLCODE          378562 non-null  int32         
 6   TAXCODE             378562 non-null  int32         
 7   TAXSUBCODE          378562 non-null  int32         
 8   OWNERCODE           378562 non-null  int32         
 9   CLASS               378562 non-null  int32         
 10  USECODE             378562 non-null  int32         
 11  LOTAREA             378562 non-null  int32         
 12  HOMESTEADFLAG       378562 non-null  bool          
 13  FARMSTEADFLAG       378562 n

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4d883e40-705b-4ee7-9326-895000a955f5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>